In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0)
plt.rcParams['image.cmap'] = 'Greys'

import numpy as np
np.set_printoptions(suppress=True)
np.set_printoptions(precision=2)

In [2]:
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)  #Modified National Institute of Standards and Technology database

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
#train set has 55000 images of size 28*28=784 and labels 1-10
print(mnist.train.images.shape)
print(mnist.train.labels.shape)

(55000, 784)
(55000, 10)


In [5]:
#test set has 10000 images of size 28*28=784 and labels 1-10
print(mnist.test.images.shape)
print(mnist.test.labels.shape)

(10000, 784)
(10000, 10)


In [7]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

def new_conv_layer(input,num_input_channels,filter_size,num_filters,use_pooling=True): 
    shape = [filter_size, filter_size, num_input_channels, num_filters]
    weights = new_weights(shape=shape)
    biases = new_biases(length=num_filters)
    layer = tf.nn.conv2d(input=input,filter=weights,strides=[1, 1, 1, 1],padding='SAME') + biases;
    if use_pooling:
        layer = tf.nn.max_pool(value=layer,ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='SAME')
    layer = tf.nn.relu(layer)
    return layer, weights

def flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat,num_features

def new_fc_layer(input,num_inputs,num_outputs,use_relu=True):
    weights = new_weights(shape=[num_inputs,num_outputs])
    biases = new_biases(length=num_outputs)
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)
    return layer

In [9]:
img_size = 28
num_channels = 1
num_classes = 10
img_size_flat = img_size * img_size * num_channels
img_shape = (img_size, img_size)

filter_size1 = 3
num_filters1 = 32

filter_size2 = 3
num_filters2 = 32

filter_size3 = 3
num_filters3 = 64

fc_size = 128 
batch_size = 32

In [11]:
x = tf.placeholder(tf.float32, [None, img_size_flat])
x_image= tf.reshape(x, [-1, img_size, img_size, num_channels])

y_true = tf.placeholder(tf.float32, shape=[None, num_classes])
y_true_cls = tf.argmax(y_true, axis=1)

In [15]:
layer_conv1, weights_conv1 = new_conv_layer(input=x_image, num_input_channels=num_channels,
                                            filter_size=filter_size1,num_filters=num_filters1,
                                            use_pooling=True)

layer_conv2, weights_conv2 = new_conv_layer(input=layer_conv1, num_input_channels=num_filters1,
                                            filter_size=filter_size2,num_filters=num_filters2,
                                            use_pooling=True)

layer_conv3, weights_conv3 = new_conv_layer(input=layer_conv2, num_input_channels=num_filters2,
                                            filter_size=filter_size3,num_filters=num_filters3,
                                            use_pooling=True)

layer_flat, num_features = flatten_layer(layer_conv3)

layer_fc1 = new_fc_layer(input=layer_flat,num_inputs=num_features,num_outputs=fc_size,use_relu=True)
layer_fc2 = new_fc_layer(input=layer_fc1,num_inputs=fc_size,num_outputs=num_classes,use_relu=False)

y_pred = tf.nn.softmax(layer_fc2)
y_pred_cls = tf.argmax(y_pred, dimension=1)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=layer_fc2,labels=y_true)
cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [20]:
session = tf.Session()
session.run(tf.initialize_all_variables())
train_batch_size = batch_size

for i in range(20000):
    batch = mnist.train.next_batch(batch_size)
    if i%100 == 0:
        train_accuracy = accuracy.eval(session=session , feed_dict = {x : batch[0], y_true : batch[1] })
        print("step %d, training accuracy %g"%(i, train_accuracy))
    optimizer.run(session=session, feed_dict={x: batch[0], y_true : batch[1]})

step 0, training accuracy 0.0625
step 100, training accuracy 0.34375
step 200, training accuracy 0.4375
step 300, training accuracy 0.78125
step 400, training accuracy 0.875
step 500, training accuracy 0.875
step 600, training accuracy 0.84375
step 700, training accuracy 0.875
step 800, training accuracy 0.90625
step 900, training accuracy 0.90625
step 1000, training accuracy 0.90625
step 1100, training accuracy 0.90625
step 1200, training accuracy 0.9375
step 1300, training accuracy 0.90625
step 1400, training accuracy 0.84375
step 1500, training accuracy 0.875
step 1600, training accuracy 0.90625
step 1700, training accuracy 0.96875
step 1800, training accuracy 0.90625
step 1900, training accuracy 0.9375
step 2000, training accuracy 0.96875
step 2100, training accuracy 1
step 2200, training accuracy 1
step 2300, training accuracy 0.9375
step 2400, training accuracy 1
step 2500, training accuracy 0.96875
step 2600, training accuracy 1
step 2700, training accuracy 0.96875
step 2800, tr

In [22]:
print(session.run(accuracy, feed_dict={x: mnist.test.images, y_true: mnist.test.labels}))

0.9864
